In [1]:
import tensorflow as tf

In [2]:
#choosing tensorflow as backend
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


In [8]:
#to configure the session 
config = tf.compat.v1.ConfigProto() 
#allocate gpu memory on runtime allocation
config.gpu_options.allow_growth=True
#to analyze how tf assigns resource, to find out which devices the operations and tensors are assigned to 
config.log_device_placement=True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)
sess

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1050 with Max-Q Design, pci bus id: 0000:02:00.0, compute capability: 6.1



In [12]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, CuDNNLSTM, LSTM, Embedding, Dropout
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.optimizers import adam

In [13]:

(X_train,Y_train),(X_test,Y_test)=imdb.load_data(num_words=20000)

In [14]:
print(len(X_train))

25000


In [15]:
X_train

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1153, 194, 82

In [16]:
print(len(X_train[0]),len(X_train[2]))

218 141


In [17]:
#padding zero infront to equal length 256
X_train_pad=pad_sequences(X_train,maxlen=256,padding='pre')
X_train_pad[4]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    1,
        249, 1323,    7,   61,  113,   10,   10,   13, 1637,   14,   20,
         56,   33, 2401,   18,  457,   88,   13, 2626, 1400,   45, 3171,
         13,   70,   79,   49,  706,  919,   13,   16,  355,  340,  355,
       1696,   96,  143,    4,   22,   32,  289,   

In [18]:
X_test_pad=pad_sequences(X_test,padding='pre',maxlen=256)

In [19]:
#Creating the model
model=Sequential()
#input dimension is total number of vocabulary , output dim is embedding size , input length is  maxlen
model.add(Embedding(input_dim=20000,output_dim=10,input_length=256))


In [20]:
#model.add(Dropout(0.2))
model.add(LSTM(128,dropout=0.2,return_sequences=False))
#model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
#model.add(CuDNNLSTM(128,return_sequences=False))

In [21]:
model.add(Dense(1,activation='sigmoid',name='classification'))

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 256, 10)           200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               71168     
_________________________________________________________________
classification (Dense)       (None, 1)                 129       
Total params: 271,297
Trainable params: 271,297
Non-trainable params: 0
_________________________________________________________________


In [96]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [98]:
callback_early_stopping=EarlyStopping(monitor='val_loss',patience=5,verbose=1)

In [103]:
%%time
model.fit(X_train_pad,Y_train,epochs=6,validation_split=0.05, callbacks=[callback_early_stopping])

Train on 23750 samples, validate on 1250 samples
Epoch 1/6
23750/23750 [==============================] - 192s 8ms/step - loss: 0.2487 - accuracy: 0.9063 - val_loss: 0.2662 - val_accuracy: 0.8960
Epoch 2/6
23750/23750 [==============================] - 244s 10ms/step - loss: 0.2176 - accuracy: 0.9192 - val_loss: 0.2738 - val_accuracy: 0.8896
Epoch 3/6
23750/23750 [==============================] - 253s 11ms/step - loss: 0.2003 - accuracy: 0.9273 - val_loss: 0.3151 - val_accuracy: 0.8864
Epoch 4/6
23750/23750 [==============================] - 233s 10ms/step - loss: 0.1831 - accuracy: 0.9319 - val_loss: 0.3596 - val_accuracy: 0.8592
Epoch 5/6
23750/23750 [==============================] - 244s 10ms/step - loss: 0.1694 - accuracy: 0.9405 - val_loss: 0.3840 - val_accuracy: 0.8864
Epoch 6/6
23750/23750 [==============================] - 239s 10ms/step - loss: 0.1587 - accuracy: 0.9439 - val_loss: 0.3234 - val_accuracy: 0.8944
Epoch 00006: early stopping
Wall time: 23min 25s


In [104]:
%%time
loss,accuracy=model.evaluate(X_test_pad,Y_test)

25000/25000 [==============================] - 98s 4ms/step
Wall time: 1min 37s


In [106]:
print(f'Loss: {loss:.2}')
print(f'Accuracy: {accuracy:.5}')

Loss: 0.33
Accuracy: 0.8788


In [122]:
model.save("..\{}".format("sentiment_LSTM"))

In [53]:
from tensorflow.keras.models import Sequential

model_GRU=Sequential()

In [54]:
from tensorflow.keras.layers import Embedding,Dense,Dropout

model_GRU.add(Embedding(input_dim=20000,output_dim=10,input_length=256,name='layer_embedding'))

In [55]:
from tensorflow.compat.v1.keras.layers import CuDNNGRU

model_GRU.add(CuDNNGRU(16,return_sequences=True))

In [56]:
model_GRU.add(CuDNNGRU(8,return_sequences=True))

In [57]:
model_GRU.add(CuDNNGRU(4,return_sequences=False))

In [58]:
model_GRU.add(Dropout(0.02))

In [59]:
model_GRU.add(Dense(1,activation='sigmoid'))

In [60]:
model_GRU.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

In [61]:
model_GRU.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 256, 10)           200000    
_________________________________________________________________
cu_dnngru_9 (CuDNNGRU)       (None, 256, 16)           1344      
_________________________________________________________________
cu_dnngru_10 (CuDNNGRU)      (None, 4)                 264       
_________________________________________________________________
dropout_2 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 201,613
Trainable params: 201,613
Non-trainable params: 0
_________________________________________________________________


In [62]:
%%time
model_GRU.fit(X_train_pad,Y_train,epochs=7,batch_size=128,validation_split=0.05)

Train on 23750 samples, validate on 1250 samples
Epoch 1/7
23750/23750 [==============================] - 9s 387us/sample - loss: 0.5793 - accuracy: 0.6887 - val_loss: 0.4191 - val_accuracy: 0.8184
Epoch 2/7
23750/23750 [==============================] - 5s 196us/sample - loss: 0.3328 - accuracy: 0.8667 - val_loss: 0.3672 - val_accuracy: 0.8496
Epoch 3/7
23750/23750 [==============================] - 5s 196us/sample - loss: 0.2550 - accuracy: 0.9045 - val_loss: 0.3525 - val_accuracy: 0.8536
Epoch 4/7
23750/23750 [==============================] - 5s 198us/sample - loss: 0.2139 - accuracy: 0.9213 - val_loss: 0.3093 - val_accuracy: 0.8736
Epoch 5/7
23750/23750 [==============================] - 5s 197us/sample - loss: 0.1769 - accuracy: 0.9365 - val_loss: 0.3545 - val_accuracy: 0.8584
Epoch 6/7
23750/23750 [==============================] - 5s 211us/sample - loss: 0.1522 - accuracy: 0.9461 - val_loss: 0.3468 - val_accuracy: 0.8792
Epoch 7/7
23750/23750 [==============================] - 

In [63]:
loss,accuracy=model_GRU.evaluate(x=X_test_pad,y=Y_test)

25000/25000 [==============================] - 12s 468us/sample - loss: 0.3713 - accuracy: 0.8705


In [51]:
print(f'Loss: {loss:.2}')
print(f'Accuracy:{accuracy:.2}')

Loss: 0.35
Accuracy:0.86
